***GENERATED CODE FOR classify PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Date', 'transformation_label': 'String Indexer'}], 'feature': 'Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '1/1/2014', 'max': '9/7/2018', 'missing': '0', 'distinct': '488'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Date'}, {'feature_label': 'Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product Category', 'transformation_label': 'String Indexer'}], 'feature': 'Product Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AI', 'max': 'OEM', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product Category'}, {'feature_label': 'Product Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product Name', 'transformation_label': 'String Indexer'}], 'feature': 'Product Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Quadro P2200', 'max': 'Tesla V100', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product Name'}, {'feature_label': 'Product Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Region', 'transformation_label': 'String Indexer'}], 'feature': 'Region', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'APAC', 'max': 'South America', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Region'}, {'feature_label': 'Region', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Region')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer Segment', 'transformation_label': 'String Indexer'}], 'feature': 'Customer Segment', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AI Researchers', 'max': 'Gamers', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer Segment'}, {'feature_label': 'Customer Segment', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer Segment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer Type', 'transformation_label': 'String Indexer'}], 'feature': 'Customer Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Business', 'max': 'Individual', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer Type'}, {'feature_label': 'Customer Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer Type')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Marketing Spend (USD)', 'threshold': 110251.168, 'transformation_label': 'Binarizer'}], 'feature': 'Marketing Spend (USD)', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '109155.91', 'stddev': '54709.68', 'min': '10187.78', 'max': '199527.94', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'Marketing Spend (USD)'}, {'feature_label': 'Marketing Spend (USD)', 'threshold': 110251.168, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('Marketing Spend (USD)')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Competitor Influence', 'transformation_label': 'String Indexer'}], 'feature': 'Competitor Influence', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'High', 'max': 'Medium', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Competitor Influence'}, {'feature_label': 'Competitor Influence', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Competitor Influence')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product Launch Date', 'transformation_label': 'String Indexer'}], 'feature': 'Product Launch Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/1/2005', 'max': '9/9/2020', 'missing': '0', 'distinct': '486'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product Launch Date'}, {'feature_label': 'Product Launch Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product Launch Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Competitor Product', 'transformation_label': 'String Indexer'}], 'feature': 'Competitor Product', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AMD Radeon RX 6800', 'max': 'Intel Xeon', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Competitor Product'}, {'feature_label': 'Competitor Product', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Competitor Product')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sales Revenue (USD)', 'threshold': 742122300.098, 'transformation_label': 'Binarizer'}], 'feature': 'Sales Revenue (USD)', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '52631269490.66', 'stddev': '169976350365.33', 'min': '2.80018815E8', 'max': '1.12E12', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sales Revenue (USD)'}, {'feature_label': 'Sales Revenue (USD)', 'threshold': 742122300.098, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('Sales Revenue (USD)')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run classifyHooks.ipynb
try:
	#sourcePreExecutionHook()

	nvidiamarketanalytics = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Nvidia_market_analytics1.csv', 'filename': 'Nvidia_market_analytics1.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(nvidiamarketanalytics)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run classifyHooks.ipynb
try:
	#transformationPreExecutionHook()

	classifyautofe = TransformationMain.run(nvidiamarketanalytics,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Date", "transformation_label": "String Indexer"}], "feature": "Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/1/2014", "max": "9/7/2018", "missing": "0", "distinct": "488"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Date"}, {"transformationsData": [{"feature_label": "Product Category", "transformation_label": "String Indexer"}], "feature": "Product Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AI", "max": "OEM", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product Category"}, {"transformationsData": [{"feature_label": "Product Name", "transformation_label": "String Indexer"}], "feature": "Product Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Quadro P2200", "max": "Tesla V100", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product Name"}, {"transformationsData": [{"feature_label": "Region", "transformation_label": "String Indexer"}], "feature": "Region", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "APAC", "max": "South America", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Region"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Units Sold", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2550.13", "stddev": "1452.84", "min": "102", "max": "4998", "missing": "0"}, "updatedLabel": "Units Sold"}, {"transformationsData": [{"feature_label": "Customer Segment", "transformation_label": "String Indexer"}], "feature": "Customer Segment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AI Researchers", "max": "Gamers", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer Segment"}, {"transformationsData": [{"feature_label": "Customer Type", "transformation_label": "String Indexer"}], "feature": "Customer Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Business", "max": "Individual", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer Type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer Satisfaction", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.98", "stddev": "1.17", "min": "1.0", "max": "5.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Customer Satisfaction"}, {"transformationsData": [{"feature_label": "Marketing Spend (USD)", "threshold": 110251.168, "transformation_label": "Binarizer"}], "feature": "Marketing Spend (USD)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "109155.91", "stddev": "54709.68", "min": "10187.78", "max": "199527.94", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "Marketing Spend (USD)"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Discount Percentage (%)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "12.56", "stddev": "7.43", "min": "0", "max": "25", "missing": "0"}, "updatedLabel": "Discount Percentage (%)"}, {"transformationsData": [{"feature_label": "Competitor Influence", "transformation_label": "String Indexer"}], "feature": "Competitor Influence", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "High", "max": "Medium", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Competitor Influence"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Return Rate (%)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.68", "stddev": "1.37", "min": "0.12", "max": "5.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Return Rate (%)"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AI/ML Adoption Rate (%)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "51.83", "stddev": "23.16", "min": "10", "max": "90", "missing": "0"}, "updatedLabel": "AI/ML Adoption Rate (%)"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ad Campaign Effectiveness", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.18", "stddev": "0.07", "min": "0.05", "max": "0.3", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ad Campaign Effectiveness"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer Retention Rate (%)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "70.39", "stddev": "12.1", "min": "50", "max": "90", "missing": "0"}, "updatedLabel": "Customer Retention Rate (..."}, {"transformationsData": [{"feature_label": "Product Launch Date", "transformation_label": "String Indexer"}], "feature": "Product Launch Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/1/2005", "max": "9/9/2020", "missing": "0", "distinct": "486"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product Launch Date"}, {"transformationsData": [{"feature_label": "Competitor Product", "transformation_label": "String Indexer"}], "feature": "Competitor Product", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AMD Radeon RX 6800", "max": "Intel Xeon", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Competitor Product"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Market Share (%)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "49.47", "stddev": "11.6", "min": "30.02", "max": "69.99", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Market Share (%)"}, {"transformationsData": [{"feature_label": "Sales Revenue (USD)", "threshold": 742122300.098, "transformation_label": "Binarizer"}], "feature": "Sales Revenue (USD)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "52631269490.66", "stddev": "169976350365.33", "min": "2.80018815E8", "max": "1.12E12", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "Sales Revenue (USD)"}]}))

	#transformationPostExecutionHook(classifyautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run classifyHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(classifyautofe, ["Units Sold", "Customer Satisfaction", "Discount Percentage (%)", "Return Rate (%)", "AI/ML Adoption Rate (%)", "Ad Campaign Effectiveness", "Customer Retention Rate (%)", "Market Share (%)", "Date_stringindexer", "Product Category_stringindexer", "Product Name_stringindexer", "Region_stringindexer", "Customer Segment_stringindexer", "Customer Type_stringindexer", "Marketing Spend (USD)_binarizer", "Competitor Influence_stringindexer", "Product Launch Date_stringindexer", "Competitor Product_stringindexer"], "Sales Revenue (USD)_binarizer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

